# Coursework: Image Warping
### Visual Computing

This notebook contains exercises related to image warping, image stitching, non-linear distortions, image morphing, and real-time panorama stitching. Bonus questions are included with specific marks indicated.

**Grading Scheme (100 points total)**:

- **Affine Transformations**: 10 points
- **Homographies**: 10 points
- **Projective Transformations**: 10 points
- **Non-linear Distortions and Image Stitching**: 15 points
- **Image Warping and Interpolation**: 25 points
- **Panorama Stitching (Real-Time)**: 15 points
- **Image Morphing (Face morphing, scene transitions, etc.)**: 15 points




### 1. Affine Transformations- Recap (10 points)
Affine transformations map points between two planes while preserving lines and parallelism, but not necessarily angles. In this task, you will explore how affine transformations can be used to transform an image geometrically.
**Task:**
- Implement affine transformations to warp an image using a 2x3 transformation matrix.
- Apply the transformation matrix to change the geometry of the image, such as rotating, scaling, translating, or shearing.
- Visualize the original image and the transformed (warped) image to see how the transformation affects the image.

In [ ]:
# Code for Affine Transformations here
# Implement the affine transformation function and visualization

# Code for Affine Transformations here
import cv2
import numpy as np
import matplotlib.pyplot as plt

image = cv2.imread("364547.png")
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

transformation_matrix = np.float32([
    [2, 0, 50], 
    [0, 2, 30]
])

# Implement the affine transformation function and visualization

def apply_affine_transformation(image, transformation_matrix):
    shape = image.shape
    rows = shape[0]
    cols = shape[1]
    transformed_image = cv2.warpAffine(image, transformation_matrix, (cols, rows))
    return transformed_image

transformed_image = apply_affine_transformation(image, transformation_matrix)

plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.imshow(image)
plt.title("Original Image")

plt.subplot(1, 2, 2)
plt.imshow(transformed_image)
plt.title("Transformed Image")

plt.show()

### 2. Homographies (5 points + 5 Bonus Points)
Homographies describe a more complex transformation that maps one plane to another, allowing for perspective effects and changes in shape that still preserve straight lines.

**Task:**
- Compute the homography matrix between two sets of corresponding points from two images.
- Apply the homography matrix to warp one image into the coordinate space of the other.
- Use feature detection (like ORB or SIFT) to automatically detect keypoints between the images and align them using the homography.

**Bonus Question (5 points):**
- Investigate how the number of corresponding points affects the accuracy of the homography. Explore what happens if fewer or more points are used and report your observations.

In [ ]:
# Code for Homographies here
# Implement homography computation and image warping

image1 = cv2.imread("image1.png", cv2.IMREAD_GRAYSCALE)
image2 = cv2.imread("image2.png", cv2.IMREAD_GRAYSCALE)

orb = cv2.ORB_create()

kp1, des1 = orb.detectAndCompute(image1, None)
kp2, des2 = orb.detectAndCompute(image2, None)

bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
matches = sorted(bf.match(des1, des2), key=lambda x: x.distance)

src_pts = np.float32([kp1[m.queryIdx].pt for m in matches])
dst_pts = np.float32([kp2[m.trainIdx].pt for m in matches])

H, _ = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC)

shape = image2.shape
rows = shape[0]
cols = shape[1]
warped_img = cv2.warpPerspective(image1, H, (cols, rows))

# Plot the results
plt.figure(figsize=(12, 6))
plt.subplot(1, 3, 1)
plt.imshow(image1, cmap='gray')


plt.subplot(1, 3, 2)
plt.imshow(image2, cmap='gray')

plt.subplot(1, 3, 3)
plt.imshow(warped_img, cmap='gray')
plt.show()


### 3. Projective Transformations (5 points + 5 Bonus Points)
Projective transformations allow us to map points from one plane to another, but with perspective distortion. This is commonly used in applications like perspective correction and image warping.

**Task:**
- Implement a projective transformation matrix and apply it to warp an image.
- Use this transformation to warp an image as if it is viewed from a different perspective.
- Experiment with multiple projective transformations to create different perspectives and analyze how the image is affected.

**Bonus Question (5 points):**
- Handle cases where the backward mapping (from the target image back to the source) falls outside the image bounds. Implement strategies like padding, extrapolation, or ignoring out-of-bounds pixels.

In [ ]:
# Code for Projective Transformations here
# Implement projective transformation and handle backward mapping

image1 = cv2.imread("364547.png")
image1 = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

shape = image.shape
rows = shape[0]
cols = shape[1]

transformation_matrix = np.float32([
    [0.5, 0.1, cols // 4],
    [0.1, 0.5, rows // 4],
    [0.001, 0.001, 1]
])

transformation_matrix_2 = np.float32([
    [0, 0.2, 100], 
    [0, 1, 50],
    [0.001, 0.002, 1]
])

warped_image = cv2.warpPerspective(image, transformation_matrix, (cols, rows))
warped_image2 = cv2.warpPerspective(image, transformation_matrix_2, (cols, rows))

plt.figure(figsize=(10, 5))

plt.subplot(1, 3, 1)
plt.title('Original Image')
plt.imshow(image)
plt.axis('off')

plt.subplot(1, 3, 2)
plt.title('Warped Image')
plt.imshow(warped_image)
plt.axis('off')

plt.subplot(1, 3, 3)
plt.title('Warped Image 2')
plt.imshow(warped_image2)
plt.axis('off')

plt.tight_layout()
plt.show()



### 4. Non-Linear Distortions and Image Stitching (15 points)
Non-linear distortions allow you to stretch, compress, or warp an image in non-uniform ways. Image stitching involves aligning multiple images and blending them together to create a larger, seamless composite.

**Task:**
- Apply non-linear distortions to an image, such as using a fisheye effect or other non-linear transformation.
- After experimenting with distortions, move to stitching two or more images together by finding corresponding points, aligning the images using a transformation, and blending them.
- For stitching, make sure to use blending techniques (such as linear blending or multi-band blending) to avoid visible seams where the images overlap.

In [ ]:
# Code for Non-Linear Distortions and Image Stitching here
# Apply non-linear distortions and implement image stitching with blending

### 5. Image Warping and Interpolation (20 points + 5 Bonus Points)
When warping images, pixel coordinates often map to non-integer values. Interpolation helps you estimate pixel values at these non-integer coordinates to avoid visual artifacts like pixelation or blurring.

**Task:**
- Implement nearest-neighbor interpolation to handle sub-pixel coordinates during image warping.
- Implement bilinear interpolation, a more advanced method, to provide smoother results.
- Compare the results of both interpolation techniques by warping the same image and observing the visual differences.

**Bonus Question (5 points):**
- How does interpolation affect the quality of stitched or warped images? Analyze how different interpolation methods can impact the final output and report your observations.

In [ ]:
# Code for Image Warping and Interpolation here
# Implement nearest-neighbor and bilinear interpolation and compare results

### 6. Panorama Stitching (Real-Time) (15 points)
Panorama stitching involves aligning and stitching together multiple images to create a single wide-angle composite. Real-time panorama stitching takes this a step further, allowing for immediate feedback and dynamic composition as more images are added.

**Task:**
- Implement real-time panorama stitching with multiple images.
- Use projective transformations to align images and blend them smoothly.
- Ensure that the stitched images have minimal visible seams and apply real-time adjustments as new images are added.

**HINT:**
 You can explore techniques like feathering, linear blending, or multi-band blending for better panaromas.

In [ ]:
# Code for Real-Time Panorama Stitching here
# Implement real-time panorama stitching and experiment with blending techniques

### 7. Image Morphing (Face Morphing, Scene Transitions) (15 points)
Image morphing is the process of smoothly transitioning from one image to another by combining both geometric warping and cross-dissolving (blending). Common applications include face morphing and smooth transitions between scenes in videos or graphics.

**Task:**
- Morph between two faces or two scenes by first geometrically warping the images so that key features align.
- Use blending techniques to create a smooth transition from one image to another. Ensure that the transition looks gradual and natural, without sudden jumps or artifacts.
- Consider experimenting with intermediate steps to control the pace of the morphing effect.

In [ ]:
# Code for Image Morphing here
# Implement image morphing between two images with warping and blending